In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from pathlib import Path

# ---------------------------
# Mann-Kendall Test Function
# ---------------------------
def mann_kendall_test(series, alpha=0.05):
    x = np.array(series)
    x = x[~np.isnan(x)]
    n = len(x)

    S = 0
    for k in range(n - 1):
        for j in range(k + 1, n):
            if x[j] > x[k]:
                S += 1
            elif x[j] < x[k]:
                S -= 1

    var_S = (n * (n - 1) * (2*n + 5)) / 18

    if S > 0:
        Z = (S - 1) / np.sqrt(var_S)
    elif S < 0:
        Z = (S + 1) / np.sqrt(var_S)
    else:
        Z = 0

    p = 2 * (1 - norm.cdf(abs(Z)))
    Z_crit = norm.ppf(1 - alpha/2)

    if abs(Z) > Z_crit:
        trend = "Increasing" if Z > 0 else "Decreasing"
    else:
        trend = "No Significant Trend"

    return S, var_S, Z, p, trend

# ---------------------------
# Process Your Excel File
# ---------------------------
file = Path(r"D:\climate change\monthly_averages.xlsx")
sheets = ["Rainfall_monthly", "Tmax_monthly", "Tmin_monthly"]   # modify if needed

results = []

for sheet in sheets:
    print(f"\n📂 Processing Sheet: {sheet}")

    df = pd.read_excel(file, sheet_name=sheet, index_col=0)
    df = df.drop(columns=["Lon", "Lat"], errors="ignore")

    for station in df.columns:
        S, varS, Z, p, trend = mann_kendall_test(df[station])

        results.append({
            "Sheet": sheet,
            "Station": station,
            "S": S,
            "Var(S)": varS,
            "Z": Z,
            "p-value": p,
            "Trend": trend
        })

# ---------------------------
# Save Output
# ---------------------------
output = Path(r"D:\climate change\MK_results.csv")
pd.DataFrame(results).to_csv(output, index=False)

print(f"\n✅ Mann-Kendall results saved to:\n{output}")



📂 Processing Sheet: Rainfall_monthly

📂 Processing Sheet: Tmax_monthly

📂 Processing Sheet: Tmin_monthly

✅ Mann-Kendall results saved to:
D:\climate change\MK_results.csv
